In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
import joblib

df = pd.read_csv("students_performance.csv")


In [2]:
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
cat_cols = df.select_dtypes(include=['object']).columns

df[num_cols].head(), df[cat_cols].head()


(        age  gender  parental_education  study_hours_week  attendance_rate  \
 0  0.175417     5.0                 8.0         -0.974905         1.029991   
 1 -1.128525     5.0                 8.0          0.715019        -0.084645   
 2  1.479359     5.0                11.0          0.080192         0.008848   
 3 -0.259230     3.0                 8.0         -1.364208         0.903257   
 4 -1.128525     3.0                 8.0         -1.289001        -0.955162   
 
    extracurricular  sleep_hours  previous_scores  tutoring  internet_quality  \
 0              5.0    -0.428905        -0.869233       3.0               6.0   
 1              5.0     0.980578        -0.772277       3.0               6.0   
 2              5.0    -0.333786         1.580030       3.0               6.0   
 3              5.0     0.003452        -0.701762       5.0               8.0   
 4              5.0     0.928696        -0.363515       3.0               8.0   
 
    family_income  health_status  fi

In [3]:
num_imputer = SimpleImputer(strategy="median")
df[num_cols] = num_imputer.fit_transform(df[num_cols])


In [4]:
cat_imputer = SimpleImputer(strategy="most_frequent")
df[cat_cols] = cat_imputer.fit_transform(df[cat_cols])

In [5]:
def cap_outliers(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    low = Q1 - 1.5 * IQR
    high = Q3 + 1.5 * IQR
    df[col] = np.where(df[col] < low, low,
               np.where(df[col] > high, high, df[col]))
    return df

for col in num_cols:
    df = cap_outliers(df, col)


In [6]:
cat_cols = df.select_dtypes(include=['object']).columns.tolist()

# Remover identificadores que não devem ser codificados
cols_to_remove = ["student_id"]
cat_cols = [c for c in cat_cols if c not in cols_to_remove]
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
encoder.fit(df[cat_cols])
df[cat_cols] = encoder.transform(df[cat_cols])



In [7]:
print(df[ordinal_cols].dtypes)
for col in ordinal_cols:
    df[col] = df[col].fillna('').astype(str)  # Converte tudo para string e preenche valores nulos com ''
    df[col] = df[col].str.strip().str.lower()  # Remove espaços e transforma para minúsculas
print("\nCategorias após padronização:")
for col in ordinal_cols:
    print(f"{col}: {df[col].unique()}")

    # 1. Defina a lista de colunas ordinais
ordinal_cols = ['internet_quality', 'health_status']

# 2. Verificando os tipos de dados nas colunas ordinal
print(df[ordinal_cols].dtypes)

# 3. Limpeza de espaços e transformação para minúsculas nas colunas de texto
for col in ordinal_cols:
    # Garantir que valores nulos ou não-string sejam convertidos para string
    df[col] = df[col].fillna('').astype(str)  # Converte tudo para string e preenche valores nulos com ''
    df[col] = df[col].str.strip().str.lower()  # Remover espaços e transformar para minúsculas

# 4. Verificando as categorias após a padronização
print("\nCategorias após padronização:")
for col in ordinal_cols:
    print(f"{col}: {df[col].unique()}")



NameError: name 'ordinal_cols' is not defined

In [ ]:
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

joblib.dump(scaler, "scaler.pkl")


['scaler.pkl']

In [ ]:
df["study_efficiency"] = df["previous_scores"] / df["study_hours_week"]
df["sleep_quality"] = df["sleep_hours"] / 8


In [ ]:
df.to_csv("students_performance.csv", index=False)
